In [8]:
import pandas as pd
import sys
sys.path.append("C:\\Users\\Tiziano Pacifico\\Desktop\\CodeTesi")
import os
os.chdir('C:\\Users\\Tiziano Pacifico\\Desktop\\TESI\\RedditDS')
import credential
import praw
import numpy as np

# Note

Partendo dai singoli blocchi di submissions ottenuti dalla scissione del dataframe originale, abbiamo ottenuto una uguale serie di dataframes con i commenti relativi ad ogni blocco di submissions.
Abbiamo iterato progressivamente sui singoli file json e per ogni json abbiamo iterato sulle singole submissions. PEr ogni submission abbiamo estrato la foresta dei commenti senza preoccuparti della posizione gerarchiaca degli stessi nella foresta in quanto la nostra analisi si concetrerà prevalentemente sul testo indipendentemente dall'ordine in cui esso compare. 
L'estrazione è stata però fatta in questo caso attraverso delle liste. Ogni lista corrisponde alla serie che verrà inserita nella rispettiva colonna del dataframe finale. Il tempo di esecuzione dello script si è rivelato molto più basso delle attese permettendoci di estrarre la quasi totalità dei commenti in meno di una giornata.
Anche per i commenti alcune verifiche a campione hanno riportato una miss rate inferiore al 5% un valore più che accettabile al fine della nostra analisi.
Qualche eccezione è stata sollevata rispetto agli autori dei commenti, alcuni di essi infatti non hanno resituito nessun autore. Il motivo di questa assenza è ancora da indagare ma anche in questo caso il miss rate è inferiore in media al 5% per cui possiamo ritenere il notro dataset come valido.

In [9]:
reddit = praw.Reddit(client_id = credential.CLIENT_ID,
                     client_secret = credential.CLIENT_SECRET,
                     user_agent = 'fetching comments: u/Ardenzio')

In [10]:
cols = ['id','body','author','created_utc','sub_id']

In [27]:
start = 51
end = 75

In [28]:
for j in range(start,end):
    print(j)
    file_path = f'splitted_df_sub\\splitted_df_sub{j}.json'
    df_submission = pd.read_json(file_path, lines = True)
    id_list = list(df_submission['id'])
    df_comments = pd.DataFrame(columns=cols)
    df_comments = df_comments.astype({'created_utc': np.int64})
        
    comments_id = []
    comments_body = []
    comments_author = []
    comments_utc = []
    comments_sub_id = []
        
    for id in id_list:
        submission = reddit.submission(id)
        submission.comments.replace_more(limit=None)
        for comment in submission.comments.list():
            comments_id.append(comment.id)
            comments_body.append(comment.body)

            if comment.author:
                if comment.author.name:
                    comments_author.append(comment.author.name)
                else:
                    comments_author.append(None)
            else:
                comments_author.append(None)

            comments_utc.append(comment.created_utc) 
            comments_sub_id.append(id)
        
    df_comments[cols[0]] = comments_id
    df_comments[cols[1]] = comments_body
    df_comments[cols[2]] = comments_author
    df_comments[cols[3]] = comments_utc
    df_comments[cols[4]] = comments_sub_id
        
    save_file = f'splitted_df_com\\splitted_df_comm{j}.json'
        
    df_comments.to_json(save_file, orient='records', lines=True)

51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
